In [ ]:
###Importing all the neccessary libraries ###

%tensorflow_version 1.x #Forcing the google collab to get tensorflow version 1.0
import numpy as np # for mathematical operations with numpy matrix
import pandas as pd #Data_Augmentation and getting the data
import warnings

from keras.utils import to_categorical #Convert to ylabels to one hot

from sklearn.metrics import f1_score,classification_report #get f1 score and classification report
import sklearn.model_selection as model_selection #For train_test Split for static training

##Keras LIBS##
from keras.models import Sequential,load_model 
from keras.layers import Dense,Flatten,Dropout,LSTM,BatchNormalization,TimeDistributed,Bidirectional,SimpleRNN,Conv1D,GRU,MaxPooling1D
from keras.optimizers import  Adam
##Keras LIBS##

from keras.callbacks import ModelCheckpoint #save the model for each checkpoint 
import matplotlib.pyplot as plt #for plots
from google.colab import drive #need to mount drive use this only if you are using google collab to train  models
from sklearn.preprocessing import StandardScaler #Not used here but can be used to get Gaussian/Normal Distribution of the data
#TODO Find out why standard scalar is giving such a low F1 score on GradeScope


###LIBRARY IMPORTING DONE ###

In [ ]:
#Mount the respective drives to retrieve data stored in the google drive
drive.mount('/content/drive')
Root_Path="/content/drive/MyDrive/" ##This is your root path :This is same for every google drive user

In [ ]:
###UTILITY class ###
class utility():
   
    """Func@1-create_time_windows
    It creates segments of data for each timesteps
    References:: book and blogs written By Jason Brownlee
    @return X array with timetseps and Y array"""
    def create_time_windows(self,sequence,n_steps):
        X, y = [],[]

        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps
            if end_ix > len(sequence):
                break
                # gather input and output parts of the pattern
            seq_x,seq_y = sequence[i:end_ix-1,:-1],sequence[end_ix-1,-1]###Data is different from the book

            X.append(seq_x)
            y.append(seq_y)
        return np.array(X), np.array(y)

    """Func2-Creates optimised version of Time windows on stacked data
    to prevent RAM overload
    @return X array with timetseps and Y array """

    def create_time_windows_optfloat(self,sequence, n_steps):
        X, y = [],[]
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps
            if end_ix > len(sequence):
                break
                # gather input and output parts of the pattern
            seq_x,seq_y = sequence[i:end_ix-1,:-1],sequence[end_ix-1,-1]###Data is different from the book

            X.append(seq_x)
            y.append(seq_y)
        print(len(X))
        X= np.asarray(X,dtype = np.float32)
        y= np.asarray(y,dtype = np.float32)
        return np.array(X),np.array(y)
                                
    """Func3-shows plots for validation and loss
    @return plots"""
    def plot_history(self,history):
        # plot loss
        plt.title('Loss')
        plt.plot(history.history['loss'], color='blue', label='train')
        plt.plot(history.history['val_loss'], color='red', label='test')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'])
        plt.show()
        
        # plot accuracy
        plt.title('Accuracy')
        plt.plot(history.history['accuracy'], color='blue', label='train')
        plt.plot(history.history['val_accuracy'], color='red', label='test')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'])
        plt.show()

    def select_training_type(self):
        training_type=0
        print("Select the training/Testing type from the map\n")
        print("For Static Training  A.1      press 1 \n")
        print("For Static Training  A.2      press 2\n")
        print("For Dynamic Training B        press 3\n")

        print("After selecting the training type press enter")
        training_type=int(input(training_type))
        return training_type
###End of class Utility ###

In [ ]:
class Data_Augmentation():
    
    def type_1(self,Train_data_path):
        print("You have selected type1 Data Augmentation \n this type uses max of 3 datas from each subject and min of 3 datas from each subject which was atcked in a csv file")
        Train_Data_df=pd.read_csv(Train_data_path)
        labels = Train_Data_df["class"].to_numpy()
        Ylabels = np.repeat(labels, 4)##Matching frequencies
        Ylabels = Ylabels[~np.isnan(Ylabels)]
        """Just chug out the extra dimensions"""
        Ylabels=np.delete(Ylabels,95735)
        Ylabels=np.delete(Ylabels,95734)
        Ylabels = Ylabels.reshape(Ylabels.shape[0], 1)

        Train_Data_df["new_labels"]=pd.DataFrame(Ylabels)
        cols=["accx","accy","accz","gyrx","gyry","gyrz","new_labels"]
        new_Data=pd.DataFrame(Train_Data_df[cols])
        print(new_Data.head(n=5))
        #########PREPARING THE DATA X #############
        trainX=new_Data.to_numpy()
        return trainX,Ylabels

    def type_2(self,Train_data_path):
        print("You have selected type2 Data Augmentation \n this type all the datas of 8 subjects are given for training")
        Train_Data_df = pd.read_csv(Train_data_path)
        labels = Train_Data_df["class"].to_numpy()
        Ylabels = np.repeat(labels, 4)
        Ylabels = Ylabels[~np.isnan(Ylabels)]
        """Just chug out the extra dimensions"""
        Ylabels = np.delete(Ylabels, 1341650)
        Ylabels = np.delete(Ylabels, 1341649)
        Ylabels=np.delete(Ylabels,1341648)
        Ylabels=np.delete(Ylabels,1341647)
        Ylabels = np.delete(Ylabels, 1341646)
        Ylabels = np.delete(Ylabels, 1341645)
        Ylabels = Ylabels.reshape(Ylabels.shape[0], 1)

        Train_Data_df["new_labels"] = pd.DataFrame(Ylabels)
        cols = ["accx", "accy", "accz", "gyrx", "gyry", "gyrz", "new_labels"]
        new_Data = pd.DataFrame(Train_Data_df[cols])
        print(new_Data.head(n=5))
        #########PREPARING THE DATA #############
        trainX = new_Data.to_numpy()
        return trainX,Ylabels
 

In [ ]:
####MODEL SELECTION CLASS ####
"""Note:Hyperparameters are choosen through concventions of kagglers and blogs ,GridsearchCV,In team discussions,TA meetings and blogs"""
class SelectModel():
        """Func@1 Creates an LSTM model with one DropOut layer
            @return model"""
        def __create_lstm_dropout_model(self,timesteps,features_acce_and_gyro,outputs):
            model=Sequential()
            model.add(LSTM(100,input_shape=(timesteps,features_acce_and_gyro)))
            model.add(Dropout(0.5))
            model.add(Dense(100,activation="relu"))
            model.add(Dense(outputs,activation="softmax"))
            model.summary()
            return model
        """Func@2 Creates and LSTM model with some hyperparameter
        tuning and adds a Batch_Normalisation layer to it
        @return model"""
        def __create_model_Batch_Norm(self,timesteps,features_acce_and_gyro,outputs):
            model=Sequential()
            model.add(LSTM(200,input_shape=(timesteps,features_acce_and_gyro)))
            model.add(BatchNormalization())
            model.add(Dense(200,activation="relu"))
            model.add(Dense(outputs,activation="softmax"))
            model.summary()
            return model
        """Func@3 Creates CNN_GRU model
        @return model""" 
        def __model_CNN_GRU(self,timesteps,features_acce_and_gyro,outputs):
            model = Sequential()
            model.add(Conv1D(filters=64, kernel_size=7,input_shape=(timesteps,features_acce_and_gyro),activation='relu'))
            model.add(MaxPooling1D(pool_size=4,padding='valid'))
            model.add(BatchNormalization())
            model.add(Conv1D(filters=64, kernel_size=7,activation='relu'))
            model.add(MaxPooling1D(pool_size=2,padding='valid'))
            model.add(BatchNormalization())
            model.add(GRU(
            units=64,
            activation='tanh',
            recurrent_activation='sigmoid',
            use_bias=True,
            kernel_initializer='glorot_uniform',
            return_sequences=True, 
            dropout=0.0,
            recurrent_dropout=0.0, #0.458 
            ))   
            model.add(GRU(
            units=64,
            activation='tanh',
            recurrent_activation='sigmoid',
            use_bias=True,
            kernel_initializer='glorot_uniform',
            return_sequences=False, 
            recurrent_dropout=0.2 
            ))
            model.add(Dropout(0.1))
            model.add(Dense(outputs, kernel_initializer='uniform',activation='softmax'))
            model.summary()
            return model

        """Func@4 Creates Simple_RNN_LSTM  model with dropout and dense and like Team 22
        @return model""" 
        def __model_RNN_LSTM(self,timesteps,features_acce_and_gyro,outputs):
            model=Sequential()
            model.add(SimpleRNN(100,return_sequences=True,input_shape=(timesteps,features_acce_and_gyro)))
            model.add(Dropout(0.5))
            model.add(Dense(32,activation="relu"))
            model.add(LSTM(100,return_sequences=True))
            model.add(Dense(32,activation='relu'))
            model.add(Dropout(0.5))
            model.add(Flatten())
            model.add(Dense(4,activation="softmax"))

            model.summary()
            return model
        
        """Func5 Creates a CNN1D only as used by Jason Brownlees
            @return model"""
        def __model_CNN1D(self,timesteps,features_acce_and_gyro,outputs):
            model = Sequential()

            # add model layers
            model.add(Conv1D(filters=64, kernel_size=7,input_shape=(timesteps,features_acce_and_gyro),activation='relu'))
            model.add(MaxPooling1D(pool_size=4,padding='valid'))
            model.add(BatchNormalization())
            model.add(Conv1D(filters=64, kernel_size=7,activation='relu'))
            model.add(MaxPooling1D(pool_size=2,padding='valid'))
            model.add(BatchNormalization())
            model.add(LSTM(100,return_sequences=False))  
            model.add(Dropout(0.1))
            model.add(Dense(outputs, kernel_initializer='uniform',activation='softmax'))
            model.summary()
            return model

        """Func5 Creates and LSTM model with some hyperparameter
        tuning and adds a Batch_Normalisation layer to it"""
        def __create_model_Batch_Norm_Bilstm(self,timesteps,features_acce_and_gyro,outputs):
            model=Sequential()
            model.add(Bidirectional(LSTM(200, return_sequences=True), input_shape=(timesteps,features_acce_and_gyro)))
            model.add(LSTM(200,return_sequences=False))
            model.add(BatchNormalization())
            model.add(Dense(200,activation="relu"))
            model.add((Dense(outputs,activation="softmax")))
            model.summary()
            return model
        
        """Func6- Creates the model for dynamic training B this is only public function for 
        dynamic training with other model architectures make the other architectures public and call in if training_type=B in the main_function()"""
        def shreedhar_LSTM2_model(self,n_timesteps,n_features,n_outputs):
            model = Sequential()
            model.add(LSTM(128, return_sequences=True , input_shape=(n_timesteps,n_features)))
            model.add(Dropout(0.5))
            model.add(LSTM(128))
            model.add(Dropout(0.5))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(n_outputs, activation='softmax'))
            model.summary()
            return model


        """Func7-This is used for tscaked Training A1 and A2 where the user can choose the model archiecture they want to train on
            return model"""
        def Select_Model_for_Training(self,timesteps,features_acce_and_gyro,outputs):
            model=0
            print("From the map select which model do you want to use for training \n")
            print("LSTM_WITH_SINGLE_LAYER_DROPOUT     press 1\n")
            print("BATCH_NORM_SINGLE_LAYER_MODEL      press 2\n")
            print("CNN_WITH_GRU                       press 3\n")
            print("RNN_LSTM                           press 4\n")
            print("CNN1D                              press 5\n")
            print("BILSTM.                            press 6\n")
            print("LSTM_2_Dropout                     press 7\n")
            print("After Selecting press enter")

            model=int(input(model))
            if(model==1):
               print("selected LSTM_WITH_SINGLE_LAYER_DROPOUT here is the architecture\n") 
               model=self.__create_lstm_dropout_model(timesteps,features_acce_and_gyro,outputs)
               return model,"LSTM_DROPOUT"
            elif(model==2):
                print("selected BATCH_NORM_SINGLE_LAYER_MODEL here is the architecture\n")
                model=self.__create_model_Batch_Norm(timesteps,features_acce_and_gyro,outputs)
                return model,"BATCH_NORM_LSTM"
            elif(model==3):
                print("selected CNN_WITH_GRU  model here is the architecture\n")
                model=self.__model_CNN_GRU(timesteps,features_acce_and_gyro,outputs)
                return model,"CNN_GRU"
            elif(model==4):
                print("selected RNN_LSTM_MODEL here is the architecture\n")
                model=self.__model_RNN_LSTM(timesteps,features_acce_and_gyro,outputs)
                return model,"RNN_LSTM"
            elif(model==5):
                print("selected CNN1D here is the architecture\n")
                model=self.__model_CNN1D(timesteps,features_acce_and_gyro,outputs)
                return model,"CNN_1D"
            elif(model==6):
                print("selected BILSTM here is the architecture\n")
                model=self.__create_model_Batch_Norm_Bilstm(timesteps,features_acce_and_gyro,outputs)
                return model,"BILSTM"
            elif(model==7):
                print("Selected Shreedhar model of LSTM_DROPout")
                model=self.shreedhar_LSTM2_model(timesteps,features_acce_and_gyro,outputs)
                return model,"SHREEDHAR_LSTM_DROPOUT"
            else:
                print("Wrong option selected\n")
                self.Select_Model_for_Training(timesteps,features_acce_and_gyro,outputs)
####MODEL SELECTION CLASS ENDS ####    

In [ ]:
print("Keeping a check on all files in your Drive Project_C1\n")
!ls "/content/drive/My Drive/ECE542/Project_C1"

We are testing out all different types of training and data augmentation done by different teams to give us the best type of results:

    Now in Training stage 
    We will be having 2 main types of Training according to the Data   
    Augmentation we have done:

    (A) Training the model on static Data - Here the data of all subjects are stacked together in a single csv file and the frequency is resampled in the data augmentation class using np.repeat():

         A.1-->Here there is a uniform distribution of data of subjects where max 
         3 datsets and min 1 dataset is taken from each subject,they were stacked 
        stacked together and resampled in Data_Augmentation.type1()

            A.2-->Here all the subject datas were taken this is bruteforce training 
            forcing the model to memorise the data of all the
            subjects:Data_Augmentation.type(2)  

    (B) Training the model on dynamic data():
        Here Data Augmentation is done for each subject indiviudally in a local
        machine and model fitting is done on each subject individually

At the end A.1 and B gave us good F1 scores of above 80 .We improved the data more by hand and went ahead with A.1 we didnt test the data training anymore with B.But you are most welcome to test it out and get results but A.2 just memorised the data and will give average F1 scores to be around 60-65,So it is better to avoid that.



In [ ]:
"""Run this cell multiple times for different models rest of the cells you can run once"""
def main_func():
    # #Use gridsearchCV here
    # epochs=[10,15,20]      #10 is good enough -->15 is overfitting
    # batch_size=[32,64,128] #32 makes it faster but 128 makes the gradient more uniform i.e slowly and surely it reaches global minima

    """Step 1: Set the paths according to your drive"""
    print("\n")
    Project_Files_path=Root_Path+"ECE542/Project_C1" #Change the string here towards your Project_Files_Path
    checkpoint_model_will_be_at=Project_Files_path+ "/Checkpoints/model_checkpoint.h5" #Change towards your own model you ant to save at each step in the checkpoint folder
    saved_weights_path=Project_Files_path+"/SavedModels/" #make a directory models in your drive and all the models after training will be saved here
    #print to make sure
    print("Your Project Filies Path is given as "+ Project_Files_path+"\n"+"The Checkpoints model will be saved at "+ checkpoint_model_will_be_at +"\n" + "The model after training will be saved at "+saved_weights_path +"\n")
    #print a WARNING
    warnings.warn("Please check if all the paths are correctly otherwise your model will not be saved at all and you may have to retrain it")
    print("\n")

    """Step2: Create class objects"""
    Utility=utility()##Create utility class object to be used later keep it as singleton object
    model=SelectModel() ##Create model class object to be used later keep it as a singleton object
    Augment=Data_Augmentation() ##The type of data augmentation you want

    """Step 3: Go for Training"""
    training_type=Utility.select_training_type()

   ###TRAINING TYPE A.1 Starts ####

    if(training_type == 1):
        model_is_saved=False ######CHANGE THIS TO FALSE IF YOU WANT TO RETRAIN THE MODEL####
        if(model_is_saved==False):
            print("Selected Training A.1")
            Train_data_path=Project_Files_path+"/xy.csv"
            trainX,trainY=Augment.type_1(Train_data_path)
            X_New,Y_New=Utility.create_time_windows_optfloat(trainX,100) #feature sets has to be optimised according to RAM left and data type 
            #Convert the Ynew into one-hot labels
            Y_labels_one_hot_encoded=to_categorical(Y_New,num_classes=4)

            ##Split train and validation sets
            ##testsets will be given prepared differently so no need to slice the array into 3 parts
            Xtrain,XVal,Ytrain,YVal=model_selection.train_test_split(X_New,Y_labels_one_hot_encoded,train_size=0.8,test_size=0.2)
            timesteps,features,outputs=Xtrain.shape[1],Xtrain.shape[2],Ytrain.shape[1]
            ###TO STOP RAM GOIUNG OVERBOARD###
            del X_New
            del Y_New
            del Y_labels_one_hot_encoded
            ###TO STOP RAM GOIUNG OVERBOARD###
            model,label=model.Select_Model_for_Training(timesteps,features,outputs)
            checkpoint = ModelCheckpoint(checkpoint_model_will_be_at, monitor='loss', verbose=1,
            save_best_only=True, mode='auto', period=1)
            model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
            history=model.fit(Xtrain,Ytrain,epochs=10,batch_size=128,validation_data=(XVal,YVal),verbose=1,callbacks=[checkpoint])
            Utility.plot_history(history)
            
            saved_weights_path=saved_weights_path+label+"_typeA.1"+".h5"
            print("The model is saved at"+saved_weights_path)
            model.save(saved_weights_path)
            del model
            
        else:
            Val_data_path=Project_Files_path+"/xy.csv"
            trainX,trainY=Augment.type_1(Val_data_path)
            X_New,Y_New=Utility.create_time_windows_optfloat(trainX,100) #feature sets has to be optimised according to RAM left and data type 
            Xtrain,XVal,Ytrain,YVal=model_selection.train_test_split(X_New,Y_New,train_size=0.8,test_size=0.2)
            model=load_model("/content/drive/MyDrive/ECE542/Project_C1/SavedModels/CNN_GRU_typeA.1.h5")
            ypred=model.predict(XVal,verbose=1)
            ypred=np.argmax(ypred, axis=1)
            print(ypred)
            yactual=YVal
            target_names = ['class 0', 'class 1', 'class 2',"class 3"]
            print(classification_report(yactual, ypred,target_names=target_names))
            del model #free the RAM

        ###TRAINING TYPE A.1 ENDS ####

   
    ###TRAINING TYPE A.2 STARTS ####
    elif(training_type ==2):
        if(model_is_saved==False):
            print("Selected Training A.2")
            Train_data_path=Project_Files_path+"/final_train_xy.csv"
            trainX,trainY=Augment.type_1(Train_data_path)
            X_New,Y_New=Utility.create_time_windows_optfloat(trainX,100) #feature sets has to be optimised according to RAM left and data type 
            #Convert the Ynew into one-hot labels
            Y_labels_one_hot_encoded=to_categorical(Y_New,num_classes=4)

            ##Split train and validation sets
            ##testsets will be given prepared differently so no need to slice the array into 3 parts
            Xtrain,XVal,Ytrain,YVal=model_selection.train_test_split(X_New,Y_labels_one_hot_encoded,train_size=0.8,test_size=0.2)
            timesteps,features,outputs=Xtrain.shape[1],Xtrain.shape[2],Ytrain.shape[1]
            ###TO STOP RAM GOIUNG OVERBOARD###
            del X_New
            del Y_New
            del Y_labels_one_hot_encoded
            ###TO STOP RAM GOIUNG OVERBOARD###
            model,label=model.Select_Model_for_Training(timesteps,features,outputs)
            checkpoint = ModelCheckpoint(checkpoint_model_will_be_at, monitor='loss', verbose=1,
            save_best_only=True, mode='auto', period=1)
            model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
            history=model.fit(Xtrain,Ytrain,epochs=10,batch_size=128,validation_data=(XVal,YVal),verbose=1,callbacks=[checkpoint])
            Utility.plot_history(history)

            saved_weights_path=saved_weights_path+label+"_typeA.2"+".h5"
            model.save(saved_weights_path)
            print("The model is saved at"+saved_weights_path)
            del model
            
        else:
            Val_data_path=Project_Files_path+"/final_train_xy.csv"
            trainX,trainY=Augment.type_1(Val_data_path)
            X_New,Y_New=Utility.create_time_windows_optfloat(trainX,100) #feature sets has to be optimised according to RAM left and data type 
            Xtrain,XVal,Ytrain,YVal=model_selection.train_test_split(X_New,Y_New,train_size=0.8,test_size=0.2)

            model=load_model("/content/drive/My Drive/ECE542/Project_C1/model.h5") ###YOU HAVE GIVE YOUR PATH TO THE MODEL SAVD NOW
            ypred=model.predict(XVal,verbose=1)
            ypred=np.argmax(ypred, axis=1)
            print(ypred)
            yactual=YVal
            target_names = ['class 0', 'class 1', 'class 2',"class 3"]
            print(classification_report(yactual, ypred,target_names=target_names))  
            del model 
        ###TRAINING TYPE A.2 ENDS ####
      
    ###TRAINING TYPE B STARTS ####
    elif(training_type==3):
        Acc=list()
        print("Training is done dynamicaly: B")
        df = pd.read_csv('/content/drive/My Drive/merged/subject_008_01__xy.csv')
        df.columns = ['0', '1', '2', '3', '4', '5', 'time', 'y', 'ytime', 'yactual']
        columns = ['0', '1', '2', '3', '4', '5', 'y']
        df = df[columns]
        Xtest, ytest = Utility.create_time_windows_optfloat(df.values, 100)
        Ytest = to_categorical(ytest, num_classes=4)
        samples = [0, 8, 5, 3, 2, 3, 3, 4, 1]
        for i in range(1, 8):
            for j in range(1, samples[i]+1):
                savepath = '/content/drive/My Drive/merged/'+'subject_00'+str(i)+'_0' + str(j) + '__xy.csv'
                
                df = pd.read_csv(savepath)
                df.columns = ['0', '1', '2', '3', '4', '5', 'time', 'y', 'ytime', 'yactual']
                columns = ['0', '1', '2', '3', '4', '5', 'y']
                df = df[columns]
                Xtrain,Ytrain=Utility.create_time_windows_optfloat(df.values,100) #feature sets has to be optimised according to RAM left and data type 
                #Convert the Ytrain into one-hot labels
                Ytrain=to_categorical(Ytrain,num_classes=4)
                print("Training model on "+ ' subject_00 '+str(i)+'_0' + str(j))
                
                timesteps,features,outputs=Xtrain.shape[1],Xtrain.shape[2],Ytrain.shape[1]
                model=model.shreedhar_LSTM2_model(timesteps,features,outputs)
                checkpoint = ModelCheckpoint(checkpoint_model_will_be_at, monitor='loss', verbose=1,
                save_best_only=True, mode='auto', period=1)
                model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
                history=model.fit(Xtrain,Ytrain,epochs=10,batch_size=32,verbose=1,callbacks=[checkpoint])

                _, accuracy = model.evaluate(Xtest, Ytest, batch_size=batch_size, verbose=1)
                print("On"+' subject_00 '+str(i)+'_0' + str(j)+"Achieved accuracy"+accuracy)
                Acc.append(accuracy)
            
        
        saved_weights_path=saved_weights_path+label+".h5"
        model.save(saved_weights_path)
        print("Accuracy of the model"+(sum(Acc)/len(Acc)))
        del model
        ###TRAINING TYPE B ENDS ####

    else:
        print("Wrong input select again")
        training_type=0;
        main_func()  


###CALL THE MAIN FUNCTION HERE###
main_func()